In [1]:
# HISTORICAL LOAD OF SIGNAL STRENGTH

# --- Signal Strength over a Date Range (Ultra Bullish / Bearish / Breakout Down) ---
# Requirements: duckdb, pandas, numpy
import duckdb, pandas as pd, numpy as np

# ====== USER SETTINGS ======
DB_PATH    = "/Users/martingobbo/stock-dashboard/data/serving/analytics.duckdb"
START_DATE = "2025-06-27"   # <-- set your start date (YYYY-MM-DD)
END_DATE   = None           # <-- set "YYYY-MM-DD" to cap the range, or None to use latest available
# ===========================

# ====== DB CONNECT ======
con = duckdb.connect(DB_PATH, read_only=True)

# ====== CONFIG — metric codes exactly matching dim_metric.metric_code ======
METRIC_CODES = [
    # Price
    'moving_avg_20d','moving_avg_50d','moving_avg_200d',
    '5_day_range_pos',
    'change_10dayret',
    'slope_over60_of_logprice','prior_slope_over60_of_logprice','60d_return_accel',
    '10_day_ret','60_day_ret','200_day_ret','300_day_ret',
    # Volume
    'abn_vol_60d',
    '60d_price_dollarVolume_correlation',
    '252d_dollar_volume_accel',
    '60d_dollar_volume_SMA','252d_dollar_volume_SMA',
    # Volatility
    '252d_upsidevolatility','252d_downsidedeviation',
    'slope_over20_of_60d_volatility','slope_over60_of_252d_volatility',
    '5d_EMA_15dayvolatility','60d_volatility',
    '60_10_highlowrange_zscore',
    # Drawdown
    '750d_drawdown',
    'drawdown_percent'
]

# ====== Helpers ======
def clamp(v, lo, hi):
    try:
        return max(lo, min(hi, v))
    except Exception:
        return lo

def nz(x):
    if x is None:
        return None
    try:
        xx = float(x)
        if np.isnan(xx):
            return None
        return xx
    except Exception:
        return None

def g(row, key):
    return nz(row.get(key))

# ====== 1) Resolve metric ids (once) ======
codes_df = pd.DataFrame({'metric_code': METRIC_CODES})
con.register('codes_df', codes_df)

met = con.execute("""
    SELECT d.metric_code, d.metric_id
    FROM dim_metric d
    JOIN codes_df c ON c.metric_code = d.metric_code
""").df()

if met.empty:
    raise RuntimeError("No metric_id found for the requested METRIC_CODES in dim_metric.")

code_to_id = dict(zip(met.metric_code, met.metric_id))
id_to_code = dict(zip(met.metric_id, met.metric_code))
metric_id_csv = ",".join(str(i) for i in met.metric_id)

# ====== 2) Determine date range ======
latest_dt = con.execute(f"""
    SELECT CAST(MAX(CAST(dt AS DATE)) AS DATE) AS max_dt
    FROM fact_metric_daily
    WHERE metric_id IN ({metric_id_csv})
""").fetchone()[0]
if latest_dt is None:
    raise RuntimeError("Could not find any dt in fact_metric_daily for the requested metrics.")

date_max = latest_dt if END_DATE is None else END_DATE

dates_df = con.execute(f"""
    SELECT DISTINCT CAST(dt AS DATE) AS d
    FROM fact_metric_daily
    WHERE metric_id IN ({metric_id_csv})
      AND CAST(dt AS DATE) >= DATE '{START_DATE}'
      AND CAST(dt AS DATE) <= DATE '{date_max}'
    ORDER BY d
""").df()

if dates_df.empty:
    raise RuntimeError(f"No dates in fact_metric_daily between {START_DATE} and {date_max} for requested metrics.")

available_dates = [str(d) for d in dates_df['d'].tolist()]

# ====== 3) Scoring functions (same as your single-day version) ======

# ---- ULTRA BULLISH ----
def gate_ultra(row):
    r10 = g(row,'10_day_ret')
    r60 = g(row,'60_day_ret')
    zAbn = g(row,'abn_vol_60d')
    corr = g(row,'60d_price_dollarVolume_correlation')
    priceOK = (r10 is not None and r10 >= 0.02) and (r60 is not None and r60 >= 0.05)
    volOK   = ((zAbn is not None and zAbn > 0.3) or (corr is not None and corr > 0.5))
    return priceOK and volOK

def score_price_ultra(row):
    pts = 0.0
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')
    if ma20 is not None and ma50 not in (None, 0):
        rel = (ma20/ma50) - 1
        if rel > 0:
            pts += 6 * clamp(rel/0.10, 0, 1)
    if ma50 is not None and ma200 not in (None, 0):
        rel = (ma50/ma200) - 1
        if rel > 0:
            pts += 6 * clamp(rel/0.10, 0, 1)
    pos = g(row,'5_day_range_pos') or 0
    pos = clamp(pos, 0, 1)
    pts += 8 * pos
    ch10 = g(row,'change_10dayret')
    if ch10 is not None and ch10 > 0:
        pts += 4
    s60  = g(row,'slope_over60_of_logprice')
    s60p = g(row,'prior_slope_over60_of_logprice')
    accel60 = g(row,'60d_return_accel')
    if (s60 is not None and s60p is not None and (s60 - s60p) > 0) or (accel60 is not None and accel60 > 0):
        pts += 3
    r10 = g(row,'10_day_ret')
    if r10 is not None and r10 > 0:
        pts += 5 * clamp(r10/0.10, 0, 1)
    r60 = g(row,'60_day_ret')
    if r60 is not None and r60 > 0:
        pts += 6 * clamp(r60/0.25, 0, 1)
    r300 = g(row,'300_day_ret')
    if r300 is not None and r300 > 0:
        pts += 4 * clamp(r300/0.50, 0, 1)
    return clamp(pts, 0, 45)

def score_volume_ultra(row):
    pts = 0.0
    z = g(row,'abn_vol_60d')
    if z is not None:
        pts += 8 * clamp(z/2, 0, 1)
    corr = g(row,'60d_price_dollarVolume_correlation')
    if corr is not None:
        pts += 7 * clamp(corr, 0, 1)
    accel252 = g(row,'252d_dollar_volume_accel')
    if accel252 is not None and accel252 > 0:
        pts += 5
    sma60, sma252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    if sma60 is not None and sma252 not in (None, 0):
        rel = (sma60/sma252) - 1
        if rel > 0:
            pts += 8 * clamp(rel/0.20, 0, 1)
    return clamp(pts, 0, 28)

def score_vola_ultra(row):
    pts = 0.0
    up252, dn252 = g(row,'252d_upsidevolatility'), g(row,'252d_downsidedeviation')
    if up252 is not None and dn252 not in (None, 0) and (up252/dn252) > 1:
        pts += 5
    slope60 = g(row,'slope_over20_of_60d_volatility')
    if slope60 is not None and slope60 <= 0:
        pts += 5
    slope252 = g(row,'slope_over60_of_252d_volatility')
    if slope252 is not None and slope252 < 0:
        pts += 4
    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    if ema15 is not None and vol60 not in (None, 0) and (ema15/vol60) < 1:
        pts += 8
    return clamp(pts, 0, 22)

def score_drawdown_ultra(row):
    pts = 0.0
    dd750 = g(row,'750d_drawdown')
    if dd750 is not None:
        pts += 2 * (1 - clamp(abs(dd750)/0.40, 0, 1))
    dd100 = g(row,'drawdown_percent')
    if dd100 is not None:
        pts += 3 * (1 - clamp(abs(dd100)/0.20, 0, 1))
    return clamp(pts, 0, 5)

# ---- BEARISH ----
def gate_bearish(row):
    r10 = g(row,'10_day_ret')
    r60 = g(row,'60_day_ret')
    zAbn = g(row,'abn_vol_60d')
    corr = g(row,'60d_price_dollarVolume_correlation')
    priceOK = (r10 is not None and r10 <= -0.02) and (r60 is not None and r60 <= -0.05)
    volOK   = ((zAbn is not None and zAbn > 0.3) or (corr is not None and corr > 0.5))
    return priceOK and volOK

def score_price_bearish(row):
    pts = 0.0
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')
    if ma20 is not None and ma50 not in (None, 0):
        rel = (ma20/ma50) - 1
        if rel < 0:
            pts += 6 * clamp((-rel)/0.10, 0, 1)
    if ma50 is not None and ma200 not in (None, 0):
        rel2 = (ma50/ma200) - 1
        if rel2 < 0:
            pts += 6 * clamp((-rel2)/0.10, 0, 1)
    pos = g(row,'5_day_range_pos') or 0
    pos = clamp(pos, 0, 1)
    pts += 8 * (1 - pos)
    ch10 = g(row,'change_10dayret')
    if ch10 is not None and ch10 < 0:
        pts += 4
    s60  = g(row,'slope_over60_of_logprice')
    s60p = g(row,'prior_slope_over60_of_logprice')
    accel60 = g(row,'60d_return_accel')
    if (s60 is not None and s60p is not None and (s60 - s60p) < 0) or (accel60 is not None and accel60 < 0):
        pts += 3
    r10 = g(row,'10_day_ret')
    if r10 is not None and r10 < 0:
        pts += 5 * clamp((-r10)/0.10, 0, 1)
    r60 = g(row,'60_day_ret')
    if r60 is not None and r60 < 0:
        pts += 6 * clamp((-r60)/0.25, 0, 1)
    r300 = g(row,'300_day_ret')
    if r300 is not None and r300 < 0:
        pts += 4 * clamp((-r300)/0.50, 0, 1)
    return clamp(pts, 0, 45)

def score_volume_bearish(row):
    pts = 0.0
    z = g(row,'abn_vol_60d')
    if z is not None:
        pts += 8 * clamp(z/2, 0, 1)
    corr = g(row,'60d_price_dollarVolume_correlation')
    if corr is not None:
        pts += 7 * clamp(-corr, 0, 1)
    accel252 = g(row,'252d_dollar_volume_accel')
    if accel252 is not None and accel252 > 0:
        pts += 5
    sma60, sma252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    if sma60 is not None and sma252 not in (None, 0):
        rel = (sma60/sma252) - 1
        if rel > 0:
            pts += 8 * clamp(rel/0.20, 0, 1)
    return clamp(pts, 0, 28)

def score_vola_bearish(row):
    pts = 0.0
    up252, dn252 = g(row,'252d_upsidevolatility'), g(row,'252d_downsidedeviation')
    if up252 is not None and dn252 not in (None, 0) and (up252/dn252) < 1:
        pts += 5
    slope60 = g(row,'slope_over20_of_60d_volatility')
    if slope60 is not None and slope60 >= 0:
        pts += 5
    slope252 = g(row,'slope_over60_of_252d_volatility')
    if slope252 is not None and slope252 > 0:
        pts += 4
    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    if ema15 is not None and vol60 not in (None, 0) and (ema15/vol60) > 1:
        pts += 8
    return clamp(pts, 0, 22)

def score_drawdown_bearish(row):
    pts = 0.0
    dd750 = g(row,'750d_drawdown')
    if dd750 is not None:
        pts += 2 * clamp(abs(dd750)/0.40, 0, 1)
    dd100 = g(row,'drawdown_percent')
    if dd100 is not None:
        pts += 3 * clamp(abs(dd100)/0.20, 0, 1)
    return clamp(pts, 0, 5)

# ---- BREAKOUT DOWN ----
def gate_breakdown(row):
    r10 = g(row,'10_day_ret')
    r60 = g(row,'60_day_ret')
    pos5 = g(row,'5_day_range_pos')
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')

    priceFresh = (r10 is not None and r10 < 0.02) and (r60 is not None and r60 > -0.10)
    nearLows = (pos5 is not None and pos5 <= 0.15)
    freshFlip = ((ma20 is not None and ma50 is not None and ma20 < ma50) or
                 (ma50 is not None and ma200 is not None and ma50 < ma200))

    zAbn = g(row,'abn_vol_60d')
    dv60, dv252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    liqUpshift = (dv60 is not None and dv252 not in (None, 0) and (dv60/dv252) >= 1.15)
    corr = g(row,'60d_price_dollarVolume_correlation')
    volConfirm = ((zAbn is not None and zAbn >= 0.5) or liqUpshift or (corr is not None and corr <= -0.2))

    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    shortVsInter = (ema15 is not None and vol60 not in (None, 0) and (ema15/vol60) > 1)

    slope20of60 = g(row,'slope_over20_of_60d_volatility')
    zRange = g(row,'60_10_highlowrange_zscore')
    dd100 = g(row,'drawdown_percent')
    volaConfirm = (
        shortVsInter
        or (slope20of60 is not None and slope20of60 >= 0)
        or (zRange is not None and zRange >= 0.25)
        or (dd100 is not None and dd100 > 0.15)
    )
    return priceFresh and nearLows and freshFlip and volConfirm and volaConfirm

def score_price_breakdown(row):
    pts = 0.0
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')
    if ma20 is not None and ma50 not in (None, 0):
        rel = (ma20/ma50) - 1
        if rel < 0:
            pts += 6 * clamp((-rel)/0.05, 0, 1)
    if ma50 is not None and ma200 not in (None, 0):
        rel2 = (ma50/ma200) - 1
        if rel2 < 0:
            pts += 6 * clamp((-rel2)/0.05, 0, 1)
    s60  = g(row,'slope_over60_of_logprice')
    s60p = g(row,'prior_slope_over60_of_logprice')
    if s60 is not None and s60p is not None:
        delta = s60 - s60p
        if delta <= 0:
            pts += 10 * clamp((-delta)/0.02, 0, 1)
    r10 = g(row,'10_day_ret')
    if r10 is not None and r10 < -0.02:
        pts += 5
    r60 = g(row,'60_day_ret')
    if r60 is not None:
        if r60 > 0:
            pts += 5
        elif r60 >= -0.10:
            pts += 2
    r200 = g(row,'200_day_ret')
    if r200 is not None:
        pts += (8 if r200 > 0 else 3)
    return clamp(pts, 0, 40)

def score_volume_breakdown(row):
    pts = 0.0
    z = g(row,'abn_vol_60d')
    if z is not None:
        pts += 10 * clamp(z/1.85, 0, 1)
    dv60, dv252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    if dv60 is not None and dv252 not in (None, 0):
        rel = (dv60/dv252) - 1
        pts += 8 * clamp(rel/0.20, 0, 1)
    corr = g(row,'60d_price_dollarVolume_correlation')
    if corr is not None:
        pts += 7 * clamp(-corr, 0, 1)
    return clamp(pts, 0, 25)

def score_vola_breakdown(row):
    pts = 0.0
    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    if ema15 is not None and vol60 not in (None, 0):
        ratio = ema15/vol60
        if ratio > 1:
            pts += 10 * clamp((ratio - 1)/0.50, 0, 1)
    slope20of60 = g(row,'slope_over20_of_60d_volatility')
    if slope20of60 is not None and slope20of60 >= 0:
        pts += 8 * clamp(slope20of60/0.02, 0, 1)
    zRange = g(row,'60_10_highlowrange_zscore')
    if zRange is not None:
        pts += 7 * clamp(zRange/1.8, 0, 1)
    return clamp(pts, 0, 25)

def score_drawdown_breakdown(row):
    pts = 0.0
    dd = g(row,'drawdown_percent')
    if dd is not None and dd <= 0.20:
        pts += 6 * (1 - clamp(abs(dd)/0.20, 0, 1))
    dd750 = g(row,'750d_drawdown')
    if dd750 is not None:
        pts += 4 * (1 - clamp(abs(dd750)/0.40, 0, 1))
    return clamp(pts, 0, 10)

# ====== 4) Iterate dates, score, collect ======
ultra_all, bearish_all, breakdown_all = [], [], []

for d in available_dates:
    vals = con.execute(f"""
        SELECT t.ticker, f.metric_id, f.value
        FROM fact_metric_daily f
        JOIN dim_ticker t USING (ticker_id)
        WHERE CAST(f.dt AS DATE) = DATE '{d}'
          AND f.metric_id IN ({metric_id_csv})
          AND t.ticker IS NOT NULL AND LENGTH(TRIM(t.ticker)) > 0
    """).df()

    if vals.empty:
        continue

    vals["metric_code"] = vals["metric_id"].map(id_to_code)
    wide = vals.pivot_table(index="ticker", columns="metric_code", values="value", aggfunc="first").reset_index()

    # Ensure all required columns exist
    for col in METRIC_CODES:
        if col not in wide.columns:
            wide[col] = np.nan

    records = wide.to_dict(orient="records")

    for r in records:
        ticker = r["ticker"]

        if gate_ultra(r):
            total = clamp(
                score_price_ultra(r) + score_volume_ultra(r) + score_vola_ultra(r) + score_drawdown_ultra(r),
                0, 100
            )
            ultra_all.append({"ticker": ticker, "signal": round(total, 3), "date": d})

        if gate_bearish(r):
            total = clamp(
                score_price_bearish(r) + score_volume_bearish(r) + score_vola_bearish(r) + score_drawdown_bearish(r),
                0, 100
            )
            bearish_all.append({"ticker": ticker, "signal": round(total, 3), "date": d})

        if gate_breakdown(r):
            total = clamp(
                score_price_breakdown(r) + score_volume_breakdown(r) + score_vola_breakdown(r) + score_drawdown_breakdown(r),
                0, 100
            )
            breakdown_all.append({"ticker": ticker, "signal": round(total, 3), "date": d})

# ====== 5) Final DataFrames across the whole period ======
ultra_signals_df    = pd.DataFrame(ultra_all).sort_values(["date","signal","ticker"], ascending=[True, False, True]).reset_index(drop=True)
bearish_signals_df  = pd.DataFrame(bearish_all).sort_values(["date","signal","ticker"], ascending=[True, False, True]).reset_index(drop=True)
breakdown_signals_df= pd.DataFrame(breakdown_all).sort_values(["date","signal","ticker"], ascending=[True, False, True]).reset_index(drop=True)

print(f"Date window: {START_DATE} → {date_max}")
print("Ultra rows:", len(ultra_signals_df))
print("Bearish rows:", len(bearish_signals_df))
print("Breakout Down rows:", len(breakdown_signals_df))

# Optional: preview heads
ultra_signals_df.head(), bearish_signals_df.head(), breakdown_signals_df.head()


Date window: 2025-06-27 → 2025-10-01
Ultra rows: 3071
Bearish rows: 1254
Breakout Down rows: 615


(  ticker  signal                 date
 0    JBL  85.323  2025-06-27 00:00:00
 1    HWM  82.126  2025-06-27 00:00:00
 2    RCL  81.095  2025-06-27 00:00:00
 3   COIN  79.378  2025-06-27 00:00:00
 4    NRG  78.034  2025-06-27 00:00:00,
   ticker  signal                 date
 0   BF.B  78.492  2025-06-27 00:00:00
 1    PCG  70.110  2025-06-27 00:00:00
 2   ENPH  69.813  2025-06-27 00:00:00
 3    DOW  68.341  2025-06-27 00:00:00
 4    CPB  67.336  2025-06-27 00:00:00,
   ticker  signal                 date
 0      K  46.561  2025-06-27 00:00:00
 1    LMT  41.430  2025-06-27 00:00:00
 2    EOG  28.335  2025-06-27 00:00:00
 3    XOM  22.807  2025-06-27 00:00:00
 4    PFE  22.582  2025-06-27 00:00:00)

In [21]:
# === DAILY PORTFOLIO (single output: daily_portfolio) =========================
import pandas as pd
from typing import Iterable, Optional, Dict, Any

# ========= CONFIG YOU CAN TUNE ========
# Choose which DataFrame supplies (ticker, signal, date)
# e.g., ultra_signals_df, bearish_signals_df, breakout_signals_df, etc.
INPUT_DF = ultra_signals_df

TOP_N = 5                  # Ignored if KEEP_ALL=True
KEEP_ALL = True           # If True, keep all tickers per day (after rules/cooldowns)
RULES: Dict[str, Any] = {
    "min_signal": 67,    # e.g., 50.0  (None disables)
    "include": None,       # e.g., {"AAPL","MSFT","NVDA"}  (None disables)
    "exclude": None,       # e.g., {"TSLA"}                (None disables)
    "cool_down_days": 0,   # e.g., 3  (0 disables)
}
# =====================================

def _normalize_dates(df: pd.DataFrame) -> pd.DataFrame:
    x = df.copy()
    x["date"] = pd.to_datetime(x["date"]).dt.date
    return x

def _apply_basic_rules(
    df: pd.DataFrame,
    min_signal: Optional[float],
    include: Optional[Iterable[str]],
    exclude: Optional[Iterable[str]],
) -> pd.DataFrame:
    x = df.copy()
    if min_signal is not None:
        x = x.loc[x["signal"] >= float(min_signal)]
    if include is not None:
        inc = set(include)
        x = x.loc[x["ticker"].isin(inc)]
    if exclude is not None:
        exc = set(exclude)
        x = x.loc[~x["ticker"].isin(exc)]
    return x

def build_daily_portfolio(
    signals_df: pd.DataFrame,
    top_n: int = 5,
    keep_all: bool = False,
    rules: Optional[Dict[str, Any]] = None,
) -> pd.DataFrame:
    """
    Returns ONE DataFrame named daily_portfolio with columns:
      - date, rank, ticker, signal
    Notes:
      - If keep_all=True, returns all tickers per day (still sorted by signal desc).
      - cool_down_days (if >0) prevents a ticker from reappearing within K prior days.
    """
    if rules is None:
        rules = {}

    df = _normalize_dates(signals_df)
    df = _apply_basic_rules(
        df,
        rules.get("min_signal"),
        rules.get("include"),
        rules.get("exclude"),
    )

    # Sort so strongest first within each day
    df = df.sort_values(["date", "signal"], ascending=[True, False])

    # Optional cool-down (walk forward day-by-day, ban recent selections)
    k = int(rules.get("cool_down_days", 0) or 0)
    if k > 0:
        kept_rows = []
        for day, chunk in df.groupby("date", sort=True):
            if kept_rows:
                kept = pd.concat(kept_rows, ignore_index=True)
                recent_days = sorted(kept["date"].unique())[-k:]
                banned = set(kept.loc[kept["date"].isin(recent_days), "ticker"])
            else:
                banned = set()
            allowed = chunk.loc[~chunk["ticker"].isin(banned)]
            kept_rows.append(allowed)
        df = pd.concat(kept_rows, ignore_index=True).sort_values(["date", "signal"], ascending=[True, False])

    # Pick rows per day
    if keep_all:
        selected = df
    else:
        selected = df.groupby("date", as_index=False).head(top_n)

    # Rank within each day (1 = strongest)
    selected = selected.copy()
    selected["rank"] = selected.groupby("date")["signal"].rank(method="first", ascending=False).astype(int)

    daily_portfolio = selected[["date", "rank", "ticker", "signal"]].sort_values(["date", "rank"])
    return daily_portfolio

# ====== RUN IT on your chosen INPUT_DF ======
daily_portfolio = build_daily_portfolio(
    INPUT_DF,
    top_n=TOP_N,
    keep_all=KEEP_ALL,
    rules=RULES,
)

# (Optional) display in notebooks
# from IPython.display import display
# display(daily_portfolio)


In [3]:
daily_portfolio.head()

,date,rank,ticker,signal
0,2025-06-27,1,JBL,85.323
1,2025-06-27,2,HWM,82.126
2,2025-06-27,3,RCL,81.095
3,2025-06-27,4,COIN,79.378
4,2025-06-27,5,NRG,78.034


In [22]:
# BACKTEST OF DAILY PORTFOLIO. WITH COLUMNS DATE AND TICKERS

from bisect import bisect_right
import duckdb, pandas as pd

# ====== SETTINGS ======
DB_PATH = "/Users/martingobbo/stock-dashboard/data/serving/analytics.duckdb"
INITIAL_CAPITAL = 100.0  # change if you want

# ====== INPUT: daily_long ======
# Expecting a pandas DataFrame `daily_long` already in memory with columns: ['date','ticker']
daily_long = daily_portfolio.copy()
daily_long['date'] = pd.to_datetime(daily_long['date']).dt.date
daily_long['ticker'] = daily_long['ticker'].astype(str).str.strip().str.upper()
daily_long = daily_long.drop_duplicates(subset=['date','ticker']).sort_values(['date','ticker'])

# Remove any stale ticker_id columns
for col in list(daily_long.columns):
    if col == 'ticker_id' or col.startswith('ticker_id') or col.endswith('.1'):
        daily_long = daily_long.drop(columns=col)

# ====== CONNECT & LOAD REFS ======
con = duckdb.connect(DB_PATH, read_only=True)

dim_ticker = con.execute("""
    SELECT ticker, ticker_id
    FROM dim_ticker
""").df()
dim_ticker['ticker'] = dim_ticker['ticker'].astype(str).str.strip().str.upper()

# ====== MAP TICKER -> TICKER_ID ======
ticker_to_id = dict(zip(dim_ticker['ticker'], dim_ticker['ticker_id']))
daily_long['ticker_id'] = daily_long['ticker'].map(ticker_to_id)

missing = sorted(daily_long.loc[daily_long['ticker_id'].isna(), 'ticker'].unique())
if missing:
    print(f"[WARN] {len(missing)} tickers not found in dim_ticker and will be skipped:")
    print(missing[:50] + (['...'] if len(missing) > 50 else []))

daily_long = (
    daily_long.dropna(subset=['ticker_id'])
              .assign(ticker_id=lambda df: df['ticker_id'].astype('int64'))
              .drop_duplicates(subset=['date','ticker_id'])
              .sort_values(['date','ticker_id'])
)

if daily_long.empty:
    con.close()
    raise ValueError("No tickers in daily_long could be mapped to dim_ticker.ticker_id. Fix tickers and retry.")

# ====== PRICE PULL (include adj_close) ======
start_dt = daily_long['date'].min()
end_dt   = daily_long['date'].max()

ticker_ids = sorted(daily_long['ticker_id'].unique().tolist())
if not ticker_ids:
    con.close()
    raise ValueError("No ticker_ids to fetch prices for (after mapping).")

ids_sql = ",".join(str(int(x)) for x in ticker_ids)

prices = con.execute(f"""
    WITH p AS (
        SELECT
            dt,
            ticker_id,
            open,
            close,
            adj_close
        FROM fact_price_daily
        WHERE ticker_id IN ({ids_sql})
    )
    SELECT *
    FROM p
    WHERE dt >= DATE '{start_dt}' - INTERVAL 2 DAY
      AND dt <= DATE '{end_dt}'   + INTERVAL 10 DAY
""").df()

con.close()

if prices.empty:
    raise ValueError("No price rows returned for the requested tickers/date range from fact_price_daily. Check contents and column names.")

# Clean price dtypes
prices['dt'] = pd.to_datetime(prices['dt']).dt.date

# ====== GLOBAL TRADING CALENDAR ======
cal = sorted(prices['dt'].unique())
if not cal:
    raise ValueError("Trading calendar is empty (no distinct dt values).")

# Fast lookup: MultiIndex
px = prices.set_index(['dt','ticker_id']).sort_index()

def get_open(d, tid):
    try:
        return float(px.loc[(d, tid), 'open'])
    except KeyError:
        return None

def get_adj_close(d, tid):
    try:
        val = px.loc[(d, tid), 'adj_close']
        return None if pd.isna(val) else float(val)
    except KeyError:
        return None

def get_close(d, tid):
    try:
        val = px.loc[(d, tid), 'close']
        return None if pd.isna(val) else float(val)
    except KeyError:
        return None

def next_trading_day(d):
    """Next dt strictly after d using calendar 'cal'."""
    i = bisect_right(cal, d)
    if i >= len(cal):
        return None
    return cal[i]

# ---------- BACKTEST STATE ----------
cash = float(INITIAL_CAPITAL)
positions = {}  # ticker_id -> {'shares': float, 'avg_cost': float, 'entry_date': date}
prior_total_value = cash

ledger_rows = []
equity_rows = []

# Pre-group signals by date -> set of ticker_ids
signals_by_date = (
    daily_long.groupby('date')['ticker_id']
    .apply(lambda s: sorted(set(s)))
    .to_dict()
)

# ---------- MAIN LOOP ----------
for t in sorted(signals_by_date.keys()):
    target_tids = signals_by_date[t]

    # Execution/valuation day
    trade_dt = next_trading_day(t)
    if trade_dt is None:
        break

    # Tradable = has valid open
    tradable = []
    for tid in target_tids:
        if get_open(trade_dt, tid) is not None:
            tradable.append(tid)

    # Current holdings
    current_held = sorted(positions.keys())

    # 1) SELL everything not in today's tradable targets
    sell_list = []
    for tid in current_held:
        if tid not in tradable:
            opx = get_open(trade_dt, tid)
            if opx is not None and positions[tid]['shares'] != 0.0:
                shares_to_sell = positions[tid]['shares']
                proceeds = shares_to_sell * opx
                cash += proceeds
                ledger_rows.append({
                    'trade_dt': trade_dt,
                    'signal_dt': t,
                    'ticker_id': tid,
                    'side': 'SELL',
                    'shares': shares_to_sell,
                    'price': opx,
                    'dollar': proceeds,
                    'reason': 'rebalance_remove'
                })
                sell_list.append(tid)

    for tid in sell_list:
        positions.pop(tid, None)

    # 2) BUY (equal weight) for tradable targets
    if len(tradable) > 0:
        base_value_for_targets = prior_total_value
        target_weight = 1.0 / len(tradable)
        target_dollars_each = base_value_for_targets * target_weight

        for tid in tradable:
            opx = get_open(trade_dt, tid)
            if opx is None or opx <= 0:
                continue
            desired_shares = target_dollars_each / opx
            current_shares = positions.get(tid, {}).get('shares', 0.0)
            delta_shares = desired_shares - current_shares
            if abs(delta_shares) > 0:
                notional = delta_shares * opx
                cash -= notional
                ledger_rows.append({
                    'trade_dt': trade_dt,
                    'signal_dt': t,
                    'ticker_id': tid,
                    'side': 'BUY' if delta_shares > 0 else 'SELL',
                    'shares': delta_shares,
                    'price': opx,
                    'dollar': notional,
                    'reason': 'rebalance_target'
                })
                positions[tid] = {
                    'shares': desired_shares,
                    'avg_cost': opx,
                    'entry_date': trade_dt
                }

    # 3) MARK at adj_close (fallback close → open)
    equity_value = 0.0
    for tid, pos in positions.items():
        px_val = get_adj_close(trade_dt, tid)
        if px_val is None:
            px_val = get_close(trade_dt, tid)
        if px_val is None:
            px_val = get_open(trade_dt, tid) or 0.0
        equity_value += pos['shares'] * px_val

    total_value = equity_value + cash
    traded_dollars = sum(abs(r['dollar']) for r in ledger_rows if r['trade_dt'] == trade_dt)
    turnover = traded_dollars / prior_total_value if prior_total_value > 0 else 0.0
    daily_ret = (total_value / prior_total_value - 1.0) if prior_total_value > 0 else 0.0

    equity_rows.append({
        'date': trade_dt,
        'equity_value': equity_value,
        'cash': cash,
        'total_value': total_value,
        'num_names': len(positions),
        'gross_turnover': turnover,
        'daily_return': daily_ret
    })

    prior_total_value = total_value

# ---------- OUTPUT DATAFRAMES ----------
transactions = pd.DataFrame(ledger_rows)
equity_curve = pd.DataFrame(equity_rows).sort_values('date').reset_index(drop=True)

# Join tickers for readability
if not transactions.empty:
    transactions = transactions.merge(dim_ticker[['ticker_id','ticker']], on='ticker_id', how='left')
    transactions = transactions[['trade_dt','signal_dt','ticker','ticker_id','side','shares','price','dollar','reason']]

final_positions = (
    pd.DataFrame([{'ticker_id': tid, **pos} for tid, pos in positions.items()])
      .merge(dim_ticker[['ticker_id','ticker']], on='ticker_id', how='left')
      [['ticker','ticker_id','shares','avg_cost','entry_date']]
    if len(positions) > 0 else pd.DataFrame(columns=['ticker','ticker_id','shares','avg_cost','entry_date'])
)

# Quick peeks
print("Equity curve (tail):")
print(equity_curve.tail(10))
print("\nTransactions (tail):")
print(transactions.tail(10))
print("\nFinal positions:")
print(final_positions)


Equity curve (tail):
          date  equity_value       cash  total_value  num_names  \
56  2025-09-18    121.237801   0.971838   122.209639          6   
57  2025-09-19    123.005568   0.304438   123.310007          7   
58  2025-09-22    123.722374   0.319477   124.041851          7   
59  2025-09-23    123.294572   0.411301   123.705873         12   
60  2025-09-24    122.381282   0.261872   122.643155         11   
61  2025-09-25    122.905773  -2.429742   120.476032         11   
62  2025-09-26    121.045727  -0.074009   120.971718          8   
63  2025-09-29    119.702545   2.111428   121.813973          3   
64  2025-09-30    123.005388  -1.089032   121.916356          4   
65  2025-10-01    127.555404 -31.302840    96.252564          5   

    gross_turnover  daily_return  
56        0.883608      0.027498  
57        0.280253      0.009004  
58        0.579970      0.005935  
59        1.000740     -0.002709  
60        0.332719     -0.008591  
61        0.539553     -0.01767

In [ ]:
#OTHER CODES

In [ ]:
# SAVE HISTORICAL SIGNALSTREGTH DF TO CSV

import os

# Save CSV in the current working directory
csv_path = os.path.join(os.getcwd(), "output.csv")

ultra_signals_df.to_csv(csv_path, index=False)
print(f"Saved DataFrame to {csv_path}")


In [ ]:
#MAX SIGNAL STRENGTH DAILY

import pandas as pd

# Assuming your DataFrame is ultra_signals_df

# Ensure 'date' is datetime type
ultra_signals_df['date'] = pd.to_datetime(ultra_signals_df['date']).dt.date

# Group by date and find max signal
daily_max = (
    ultra_signals_df
    .groupby('date', as_index=False)['signal']
    .max()
    .rename(columns={'signal': 'max_signal'})
)

# Display a scrollable table in Jupyter
from IPython.display import display
pd.set_option("display.max_rows", 200)  # adjust number of rows shown at once
display(daily_max)


In [ ]:
# COUNT HOW MANY TICKERS IN THE SIGNAL DAILY

# Assuming your DataFrame is ultra_signals_df

# Ensure 'date' is datetime type
ultra_signals_df['date'] = pd.to_datetime(ultra_signals_df['date']).dt.date

# Group by date and count signals
daily_count = (
    ultra_signals_df
    .groupby('date', as_index=False)['signal']
    .count()
    .rename(columns={'signal': 'count_signal'})
)

# Display a scrollable table in Jupyter
from IPython.display import display
pd.set_option("display.max_rows", 200)  # adjust number of rows shown at once
display(daily_count)


In [ ]:
#ONE DAY VERSION TO GET DAILY SIGNAL STRENGTH

# --- Signal Strength Replication in Python (Ultra Bullish / Bearish / Breakout Down) ---
# Requirements: duckdb, pandas, numpy
import duckdb, pandas as pd, numpy as np

# ====== DB CONNECT ======
db_path = "/Users/martingobbo/stock-dashboard/data/serving/analytics.duckdb"
con = duckdb.connect(db_path)

# ====== CONFIG — metric codes exactly matching dim_metric.metric_code ======
METRIC_CODES = [
    # Price
    'moving_avg_20d','moving_avg_50d','moving_avg_200d',
    '5_day_range_pos',
    'change_10dayret',
    'slope_over60_of_logprice','prior_slope_over60_of_logprice','60d_return_accel',
    '10_day_ret','60_day_ret','200_day_ret','300_day_ret',
    # Volume
    'abn_vol_60d',
    '60d_price_dollarVolume_correlation',
    '252d_dollar_volume_accel',
    '60d_dollar_volume_SMA','252d_dollar_volume_SMA',
    # Volatility
    '252d_upsidevolatility','252d_downsidedeviation',
    'slope_over20_of_60d_volatility','slope_over60_of_252d_volatility',
    '5d_EMA_15dayvolatility','60d_volatility',
    '60_10_highlowrange_zscore',
    # Drawdown
    '750d_drawdown',
    'drawdown_percent'
]

# ====== Helpers ======
def clamp(v, lo, hi):
    try:
        return max(lo, min(hi, v))
    except Exception:
        return lo

def nz(x):
    if x is None:
        return None
    try:
        xx = float(x)
        if np.isnan(xx):
            return None
        return xx
    except Exception:
        return None

# ====== 1) Resolve metric ids (no fragile f-strings) and latest date ======
codes_df = pd.DataFrame({'metric_code': METRIC_CODES})
con.register('codes_df', codes_df)

met = con.execute("""
    SELECT d.metric_code, d.metric_id
    FROM dim_metric d
    JOIN codes_df c ON c.metric_code = d.metric_code
""").df()

if met.empty:
    raise RuntimeError("No metric_id found for the requested METRIC_CODES in dim_metric.")

code_to_id = dict(zip(met.metric_code, met.metric_id))
id_to_code = dict(zip(met.metric_id, met.metric_code))
metric_id_csv = ",".join(str(i) for i in met.metric_id)

latest_dt = con.execute(f"""
    SELECT CAST(MAX(CAST(dt AS DATE)) AS DATE) AS max_dt
    FROM fact_metric_daily
    WHERE metric_id IN ({metric_id_csv})
""").fetchone()[0]

if latest_dt is None:
    raise RuntimeError("Could not find latest dt in fact_metric_daily for the requested metrics.")

# ====== 2) Pull latest rows and pivot to wide by metric_code ======
vals = con.execute(f"""
    SELECT t.ticker, f.metric_id, f.value
    FROM fact_metric_daily f
    JOIN dim_ticker t USING (ticker_id)
    WHERE CAST(f.dt AS DATE) = DATE '{latest_dt}'
      AND f.metric_id IN ({metric_id_csv})
      AND t.ticker IS NOT NULL AND LENGTH(TRIM(t.ticker)) > 0
""").df()

if vals.empty:
    raise RuntimeError(f"No fact_metric_daily rows on {latest_dt} for requested metrics.")

vals["metric_code"] = vals["metric_id"].map(id_to_code)
latest_wide = vals.pivot_table(
    index="ticker", columns="metric_code", values="value", aggfunc="first"
).reset_index()

# Ensure all needed columns exist (fill missing with NaN)
for col in METRIC_CODES:
    if col not in latest_wide.columns:
        latest_wide[col] = np.nan

def g(row, key):
    return nz(row.get(key))

# ====== 3) SCORING FUNCTIONS — mirror the JS logic ======

# ---- ULTRA BULLISH ----
def gate_ultra(row):
    r10 = g(row,'10_day_ret')
    r60 = g(row,'60_day_ret')
    zAbn = g(row,'abn_vol_60d')
    corr = g(row,'60d_price_dollarVolume_correlation')
    priceOK = (r10 is not None and r10 >= 0.02) and (r60 is not None and r60 >= 0.05)
    volOK   = ((zAbn is not None and zAbn > 0.3) or (corr is not None and corr > 0.5))
    return priceOK and volOK

def score_price_ultra(row):
    pts = 0.0
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')
    if ma20 is not None and ma50 not in (None, 0):
        rel = (ma20/ma50) - 1
        if rel > 0:
            pts += 6 * clamp(rel/0.10, 0, 1)
    if ma50 is not None and ma200 not in (None, 0):
        rel = (ma50/ma200) - 1
        if rel > 0:
            pts += 6 * clamp(rel/0.10, 0, 1)
    pos = g(row,'5_day_range_pos') or 0
    pos = clamp(pos, 0, 1)
    pts += 8 * pos
    ch10 = g(row,'change_10dayret')
    if ch10 is not None and ch10 > 0:
        pts += 4
    s60  = g(row,'slope_over60_of_logprice')
    s60p = g(row,'prior_slope_over60_of_logprice')
    accel60 = g(row,'60d_return_accel')
    if (s60 is not None and s60p is not None and (s60 - s60p) > 0) or (accel60 is not None and accel60 > 0):
        pts += 3
    r10 = g(row,'10_day_ret')
    if r10 is not None and r10 > 0:
        pts += 5 * clamp(r10/0.10, 0, 1)
    r60 = g(row,'60_day_ret')
    if r60 is not None and r60 > 0:
        pts += 6 * clamp(r60/0.25, 0, 1)
    r300 = g(row,'300_day_ret')
    if r300 is not None and r300 > 0:
        pts += 4 * clamp(r300/0.50, 0, 1)
    return clamp(pts, 0, 45)

def score_volume_ultra(row):
    pts = 0.0
    z = g(row,'abn_vol_60d')
    if z is not None:
        pts += 8 * clamp(z/2, 0, 1)
    corr = g(row,'60d_price_dollarVolume_correlation')
    if corr is not None:
        pts += 7 * clamp(corr, 0, 1)
    accel252 = g(row,'252d_dollar_volume_accel')
    if accel252 is not None and accel252 > 0:
        pts += 5
    sma60, sma252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    if sma60 is not None and sma252 not in (None, 0):
        rel = (sma60/sma252) - 1
        if rel > 0:
            pts += 8 * clamp(rel/0.20, 0, 1)
    return clamp(pts, 0, 28)

def score_vola_ultra(row):
    pts = 0.0
    up252, dn252 = g(row,'252d_upsidevolatility'), g(row,'252d_downsidedeviation')
    if up252 is not None and dn252 not in (None, 0) and (up252/dn252) > 1:
        pts += 5
    slope60 = g(row,'slope_over20_of_60d_volatility')
    if slope60 is not None and slope60 <= 0:
        pts += 5
    slope252 = g(row,'slope_over60_of_252d_volatility')
    if slope252 is not None and slope252 < 0:
        pts += 4
    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    if ema15 is not None and vol60 not in (None, 0) and (ema15/vol60) < 1:
        pts += 8
    return clamp(pts, 0, 22)

def score_drawdown_ultra(row):
    pts = 0.0
    dd750 = g(row,'750d_drawdown')
    if dd750 is not None:
        pts += 2 * (1 - clamp(abs(dd750)/0.40, 0, 1))
    dd100 = g(row,'drawdown_percent')
    if dd100 is not None:
        pts += 3 * (1 - clamp(abs(dd100)/0.20, 0, 1))
    return clamp(pts, 0, 5)

# ---- BEARISH ----
def gate_bearish(row):
    r10 = g(row,'10_day_ret')
    r60 = g(row,'60_day_ret')
    zAbn = g(row,'abn_vol_60d')
    corr = g(row,'60d_price_dollarVolume_correlation')
    priceOK = (r10 is not None and r10 <= -0.02) and (r60 is not None and r60 <= -0.05)
    volOK   = ((zAbn is not None and zAbn > 0.3) or (corr is not None and corr > 0.5))
    return priceOK and volOK

def score_price_bearish(row):
    pts = 0.0
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')
    if ma20 is not None and ma50 not in (None, 0):
        rel = (ma20/ma50) - 1
        if rel < 0:
            pts += 6 * clamp((-rel)/0.10, 0, 1)
    if ma50 is not None and ma200 not in (None, 0):
        rel2 = (ma50/ma200) - 1
        if rel2 < 0:
            pts += 6 * clamp((-rel2)/0.10, 0, 1)
    pos = g(row,'5_day_range_pos') or 0
    pos = clamp(pos, 0, 1)
    pts += 8 * (1 - pos)
    ch10 = g(row,'change_10dayret')
    if ch10 is not None and ch10 < 0:
        pts += 4
    s60  = g(row,'slope_over60_of_logprice')
    s60p = g(row,'prior_slope_over60_of_logprice')
    accel60 = g(row,'60d_return_accel')
    if (s60 is not None and s60p is not None and (s60 - s60p) < 0) or (accel60 is not None and accel60 < 0):
        pts += 3
    r10 = g(row,'10_day_ret')
    if r10 is not None and r10 < 0:
        pts += 5 * clamp((-r10)/0.10, 0, 1)
    r60 = g(row,'60_day_ret')
    if r60 is not None and r60 < 0:
        pts += 6 * clamp((-r60)/0.25, 0, 1)
    r300 = g(row,'300_day_ret')
    if r300 is not None and r300 < 0:
        pts += 4 * clamp((-r300)/0.50, 0, 1)
    return clamp(pts, 0, 45)

def score_volume_bearish(row):
    pts = 0.0
    z = g(row,'abn_vol_60d')
    if z is not None:
        pts += 8 * clamp(z/2, 0, 1)
    corr = g(row,'60d_price_dollarVolume_correlation')
    if corr is not None:
        pts += 7 * clamp(-corr, 0, 1)
    accel252 = g(row,'252d_dollar_volume_accel')
    if accel252 is not None and accel252 > 0:
        pts += 5
    sma60, sma252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    if sma60 is not None and sma252 not in (None, 0):
        rel = (sma60/sma252) - 1
        if rel > 0:
            pts += 8 * clamp(rel/0.20, 0, 1)
    return clamp(pts, 0, 28)

def score_vola_bearish(row):
    pts = 0.0
    up252, dn252 = g(row,'252d_upsidevolatility'), g(row,'252d_downsidedeviation')
    if up252 is not None and dn252 not in (None, 0) and (up252/dn252) < 1:
        pts += 5
    slope60 = g(row,'slope_over20_of_60d_volatility')
    if slope60 is not None and slope60 >= 0:
        pts += 5
    slope252 = g(row,'slope_over60_of_252d_volatility')
    if slope252 is not None and slope252 > 0:
        pts += 4
    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    if ema15 is not None and vol60 not in (None, 0) and (ema15/vol60) > 1:
        pts += 8
    return clamp(pts, 0, 22)

def score_drawdown_bearish(row):
    pts = 0.0
    dd750 = g(row,'750d_drawdown')
    if dd750 is not None:
        pts += 2 * clamp(abs(dd750)/0.40, 0, 1)
    dd100 = g(row,'drawdown_percent')
    if dd100 is not None:
        pts += 3 * clamp(abs(dd100)/0.20, 0, 1)
    return clamp(pts, 0, 5)

# ---- BREAKOUT DOWN ----
def gate_breakdown(row):
    r10 = g(row,'10_day_ret')
    r60 = g(row,'60_day_ret')
    pos5 = g(row,'5_day_range_pos')
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')

    priceFresh = (r10 is not None and r10 < 0.02) and (r60 is not None and r60 > -0.10)
    nearLows = (pos5 is not None and pos5 <= 0.15)
    freshFlip = ((ma20 is not None and ma50 is not None and ma20 < ma50) or
                 (ma50 is not None and ma200 is not None and ma50 < ma200))

    zAbn = g(row,'abn_vol_60d')
    dv60, dv252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    liqUpshift = (dv60 is not None and dv252 not in (None, 0) and (dv60/dv252) >= 1.15)
    corr = g(row,'60d_price_dollarVolume_correlation')
    volConfirm = ((zAbn is not None and zAbn >= 0.5) or liqUpshift or (corr is not None and corr <= -0.2))

    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    shortVsInter = (ema15 is not None and vol60 not in (None, 0) and (ema15/vol60) > 1)

    slope20of60 = g(row,'slope_over20_of_60d_volatility')
    zRange = g(row,'60_10_highlowrange_zscore')
    dd100 = g(row,'drawdown_percent')
    volaConfirm = (
        shortVsInter
        or (slope20of60 is not None and slope20of60 >= 0)
        or (zRange is not None and zRange >= 0.25)
        or (dd100 is not None and dd100 > 0.15)
    )
    return priceFresh and nearLows and freshFlip and volConfirm and volaConfirm

def score_price_breakdown(row):
    pts = 0.0
    ma20, ma50, ma200 = g(row,'moving_avg_20d'), g(row,'moving_avg_50d'), g(row,'moving_avg_200d')
    if ma20 is not None and ma50 not in (None, 0):
        rel = (ma20/ma50) - 1
        if rel < 0:
            pts += 6 * clamp((-rel)/0.05, 0, 1)
    if ma50 is not None and ma200 not in (None, 0):
        rel2 = (ma50/ma200) - 1
        if rel2 < 0:
            pts += 6 * clamp((-rel2)/0.05, 0, 1)
    s60  = g(row,'slope_over60_of_logprice')
    s60p = g(row,'prior_slope_over60_of_logprice')
    if s60 is not None and s60p is not None:
        delta = s60 - s60p
        if delta <= 0:
            pts += 10 * clamp((-delta)/0.02, 0, 1)
    r10 = g(row,'10_day_ret')
    if r10 is not None and r10 < -0.02:
        pts += 5
    r60 = g(row,'60_day_ret')
    if r60 is not None:
        if r60 > 0:
            pts += 5
        elif r60 >= -0.10:
            pts += 2
    r200 = g(row,'200_day_ret')
    if r200 is not None:
        pts += (8 if r200 > 0 else 3)
    return clamp(pts, 0, 40)

def score_volume_breakdown(row):
    pts = 0.0
    z = g(row,'abn_vol_60d')
    if z is not None:
        pts += 10 * clamp(z/1.85, 0, 1)
    dv60, dv252 = g(row,'60d_dollar_volume_SMA'), g(row,'252d_dollar_volume_SMA')
    if dv60 is not None and dv252 not in (None, 0):
        rel = (dv60/dv252) - 1
        pts += 8 * clamp(rel/0.20, 0, 1)
    corr = g(row,'60d_price_dollarVolume_correlation')
    if corr is not None:
        pts += 7 * clamp(-corr, 0, 1)
    return clamp(pts, 0, 25)

def score_vola_breakdown(row):
    pts = 0.0
    ema15, vol60 = g(row,'5d_EMA_15dayvolatility'), g(row,'60d_volatility')
    if ema15 is not None and vol60 not in (None, 0):
        ratio = ema15/vol60
        if ratio > 1:
            pts += 10 * clamp((ratio - 1)/0.50, 0, 1)
    slope20of60 = g(row,'slope_over20_of_60d_volatility')
    if slope20of60 is not None and slope20of60 >= 0:
        pts += 8 * clamp(slope20of60/0.02, 0, 1)
    zRange = g(row,'60_10_highlowrange_zscore')
    if zRange is not None:
        pts += 7 * clamp(zRange/1.8, 0, 1)
    return clamp(pts, 0, 25)

def score_drawdown_breakdown(row):
    pts = 0.0
    dd = g(row,'drawdown_percent')
    if dd is not None and dd <= 0.20:
        pts += 6 * (1 - clamp(abs(dd)/0.20, 0, 1))
    dd750 = g(row,'750d_drawdown')
    if dd750 is not None:
        pts += 4 * (1 - clamp(abs(dd750)/0.40, 0, 1))
    return clamp(pts, 0, 10)

# ====== 4) Apply scoring to each ticker ======
records = latest_wide.to_dict(orient="records")

ultra_rows, bearish_rows, breakdown_rows = [], [], []

for r in records:
    ticker = r["ticker"]

    if gate_ultra(r):
        total = clamp(
            score_price_ultra(r) + score_volume_ultra(r) + score_vola_ultra(r) + score_drawdown_ultra(r),
            0, 100
        )
        ultra_rows.append({"ticker": ticker, "signal": round(total, 3)})

    if gate_bearish(r):
        total = clamp(
            score_price_bearish(r) + score_volume_bearish(r) + score_vola_bearish(r) + score_drawdown_bearish(r),
            0, 100
        )
        bearish_rows.append({"ticker": ticker, "signal": round(total, 3)})

    if gate_breakdown(r):
        total = clamp(
            score_price_breakdown(r) + score_volume_breakdown(r) + score_vola_breakdown(r) + score_drawdown_breakdown(r),
            0, 100
        )
        breakdown_rows.append({"ticker": ticker, "signal": round(total, 3)})

# ====== 5) DataFrames (sorted by signal desc) ======
ultra_df = pd.DataFrame(ultra_rows).sort_values(["signal","ticker"], ascending=[False, True]).reset_index(drop=True)
bearish_df = pd.DataFrame(bearish_rows).sort_values(["signal","ticker"], ascending=[False, True]).reset_index(drop=True)
breakdown_df = pd.DataFrame(breakdown_rows).sort_values(["signal","ticker"], ascending=[False, True]).reset_index(drop=True)

print(f"Latest metric date: {latest_dt}")
print("Ultra Bullish matches:", len(ultra_df))
print("Bearish matches:", len(bearish_df))
print("Breakout Down matches:", len(breakdown_df))

# Keep raw wide metrics as well (preview heads)
latest_wide_head = latest_wide.head()
ultra_head = ultra_df.head()
bearish_head = bearish_df.head()
breakdown_head = breakdown_df.head()

latest_wide_head, ultra_head, bearish_head, breakdown_head
